## Preprocess dataset for the system introduced in https://www.aclweb.org/anthology/W19-2715/

In [ ]:
! pip install -U git+https://github.com/IINemo/isanlp.git@discourse

### 1. Prepare dataset for model training and evaluation 

In [ ]:
from utils.file_reading import read_edus

In [ ]:
symbol_map = {
    'x': 'х',
    'X': 'X',
    'y': 'у',
    '—': '-',
    '“': '«',
    '‘': '«',
    '”': '»',
    '’': '»',
    '😆': '😄',
    '😊': '😄',
    '😑': '😄',
    '😔': '😄',
    '😉': '😄',
    '❗': '😄',
    '🤔': '😄',
    '😅': '😄',
    '⚓': '😄',
    'ε': 'α',
    'ζ': 'α',
    'η': 'α',
    'μ': 'α',
    'δ': 'α',
    'λ': 'α',
    'ν': 'α',
    'β': 'α',
    'γ': 'α',
    'ν': 'α',
    'と': '尋',
    'の': '尋',
    '神': '尋',
    '隠': '尋',
    'し': '尋',
    'ё': 'е',
    'Ё': 'Е',
    '<': 'менее',
    '>': 'более'
}
# for i in range(127744, 128592):
#     symbol_map[chr(i)] = chr(128512)
    
# for i in range(9312, 10176):
#     symbol_map[chr(i)] = chr(128512)
    
# for i in range(913, 970):
#     symbol_map[chr(i)] = 'α'

def prepare_token(token):
    for key, value in symbol_map.items():
        token = token.replace(key, value)
        
    return token

In [ ]:
def annot2tags(annot, edus):
    tags = []
    cursor = 0

#     for i, edu in enumerate(edus):
#         if prepare_token(edu).find(prepare_token(annot['text'][annot['tokens'][0].begin:annot['tokens'][0].end])) != -1:
#             cursor = i         
    
    for sentence in range(len(annot['sentences'])):
        sentence_tags = []
        previous_first_token = 0
        previous_edu = ''

        for token in range(annot['sentences'][sentence].begin, annot['sentences'][sentence].end):

            if cursor == len(edus):
                is_first_token = False

            else:
                is_first_token = False
                
                tmp_edu = prepare_token(edus[cursor])
                annot['text'] = annot['text'].replace('билие ', 'Обилие ')
                original_text = annot['text'][annot['tokens'][token].begin:annot['tokens'][token].end]
                original_text = prepare_token(original_text).strip()

                if tmp_edu.startswith(original_text):
                    if previous_edu:
                        if prepare_token(annot['text'][annot['tokens'][previous_first_token].begin:annot['tokens'][
                        token].begin].strip()) == previous_edu or original_text.lower() in ["сначала", "кватахеви", 
                                                                                           "целый", "для", 
                                                                                           "максимальный", "тогда",
                                                                                           "два", "исследованием",
                                                                                            "хотя", "хоть",
                                                                                            "обилие", "активное",
                                                                                            "менее"
                                                                                           ] or tmp_edu in [
                            "и два внешних кольца (α, α).",
                            "У этого кольца самый высокий эксцентриситет из всех,",
                            prepare_token("Средний размер частичек в этом кольце 0,2— 20 метров,"),
                            "(Перевод Нгуен Тхи Тхуи Чам))",
                            "в Чувашии состоялся необъявленный праздник саха культуры.",
                            "и выпустила сборник якутской поэзии «Жемчужина Сахи» (Саха ахахӗ, 1996),",
                            "Однако и песня ему кажется какой-то бесцветной. Видимо, потому, что она творится только голосом, а не душой [Там же];",
                            "и может использоваться, как и все конструкции, содержащие турцизмы,",
                            prepare_token("Новобранщине - солдатчине, Эполетщине - бобёрщине, Всей пехотщине, поморщине, менее Хлеборобщине, военщине, Что с армейской долей венчаны более "),
                            "который фильтрует вредные ультрафиолетовые лучи Солнца.",
                            "а также крем-уход для волос «Пептиды шелка и иранская хна».",
                            "К 2010 году около 100 озоноразрушающих веществ, включая ХФУ, будут сняты с производства повсеместно."
                        ]:
                            is_first_token = True
                            previous_first_token = token
                            previous_edu = tmp_edu
                            cursor += 1
                    else:
                        is_first_token = True
                        previous_first_token = token
                        previous_edu = tmp_edu
                        cursor += 1

            tag = 'BeginSeg=Yes' if is_first_token else '_'
            sentence_tags.append(tag)

        tags.append(sentence_tags)

    return tags

In [ ]:
from isanlp.utils.annotation_conll_converter import AnnotationCONLLConverter

converter = AnnotationCONLLConverter()

In [ ]:
from isanlp.annotation import Token, Sentence


def split_by_paragraphs(annot_text, annot_tokens, annot_sentences, annot_lemma, annot_morph, annot_postag, annot_ud_postag,
                 annot_syntax_dep_tree):

        def split_on_two(sents, boundary):
            list_sum = lambda l: sum([len(sublist) for sublist in l])

            i = 1
            while list_sum(sents[:i]) < boundary and i < len(sents):
                i += 1

            intersentence_boundary = min(len(sents[i - 1]), boundary - list_sum(sents[:i - 1]))
            return (sents[:i - 1] + [sents[i - 1][:intersentence_boundary]], 
                    [sents[i - 1][intersentence_boundary:]] + sents[i:])
        
        def recount_sentences(chunk):
            sentences = []
            lemma = []
            morph = []
            postag = []
            ud_postag = []
            syntax_dep_tree = []
            tokens_cursor = 0
            local_cursor = 0

            for i, sent in enumerate(chunk['syntax_dep_tree']):
                if len(sent) > 0:
                    sentences.append(Sentence(tokens_cursor, tokens_cursor + len(sent)))
                    lemma.append(chunk['lemma'][i])
                    morph.append(chunk['morph'][i])
                    postag.append(chunk['postag'][i])
                    ud_postag.append(chunk['ud_postag'][i])
                    syntax_dep_tree.append(chunk['syntax_dep_tree'][i])
                    tokens_cursor += len(sent)

            chunk['sentences'] = sentences
            chunk['lemma'] = lemma
            chunk['morph'] = morph
            chunk['postag'] = postag
            chunk['ud_postag'] = ud_postag
            chunk['syntax_dep_tree'] = syntax_dep_tree
            
            return chunk

        chunks = []
        prev_right_boundary = -1

        for i, token in enumerate(annot_tokens[:-1]):

            if '\n' in annot_text[token.end:annot_tokens[i + 1].begin]:
                if prev_right_boundary > -1:
                    chunk = {
                        'text': annot_text[annot_tokens[prev_right_boundary].end:token.end + 1].strip(),
                        'tokens': annot_tokens[prev_right_boundary + 1:i + 1]
                    }
                else:
                    chunk = {
                        'text': annot_text[:token.end + 1].strip(),
                        'tokens': annot_tokens[:i + 1]
                    }

                lemma, annot_lemma = split_on_two(annot_lemma, i - prev_right_boundary)
                morph, annot_morph = split_on_two(annot_morph, i - prev_right_boundary)
                postag, annot_postag = split_on_two(annot_postag, i - prev_right_boundary)
                ud_postag, annot_ud_postag = split_on_two(annot_ud_postag, i - prev_right_boundary)
                syntax_dep_tree, annot_syntax_dep_tree = split_on_two(annot_syntax_dep_tree, i - prev_right_boundary)

                chunk.update({
                    'lemma': lemma,
                    'morph': morph,
                    'postag': postag,
                    'ud_postag': ud_postag,
                    'syntax_dep_tree': syntax_dep_tree,
                })
                chunks.append(recount_sentences(chunk))

                prev_right_boundary = i  # number of last token in the last chunk

        chunk = {
            'text': annot_text[annot_tokens[prev_right_boundary].end:].strip(),
            'tokens': annot_tokens[prev_right_boundary + 1:],
            'lemma' : annot_lemma,
            'morph': annot_morph,
            'postag': annot_postag,
            'ud_postag': annot_ud_postag,
            'syntax_dep_tree': annot_syntax_dep_tree,
        }
        
        chunks.append(recount_sentences(chunk))
        return chunks

In [ ]:
from glob import glob
from tqdm.autonotebook import tqdm
from utils.file_reading import read_annotation, read_edus
import re
from utils.train_test_split import split_train_dev_test

train, dev, test = split_train_dev_test('./data')
TRAIN_FILE = 'rus.rst.rrt_train.conll'
DEV_FILE = 'rus.rst.rrt_dev.conll'
TEST_FILE = 'rus.rst.rrt_test.conll'
MAX_LEN = 230


def preprocess(files, train=True, dev=False):
    print(f'preprocess {"train" if train else "test"} set')

    output_file = DEV_FILE if dev else TRAIN_FILE if train else TEST_FILE
    with open(output_file, 'w') as fo:
        for filename in tqdm(files):
            filename = filename.replace('.edus', '')
            annot = read_annotation(filename)  # split as well  ToDO:
            edus = read_edus(filename)
            last_edu = 0
            # tags = annot2tags(annot, edus)

            for i, chunk in enumerate(split_by_paragraphs(  # self,
                    annot['text'],
                    annot['tokens'],
                    annot['sentences'],
                    annot['lemma'],
                    annot['morph'],
                    annot['postag'],
                    annot['ud_postag'],
                    annot['syntax_dep_tree'])):

                sentence = 0
                token = 0
                chunk['text'] = annot['text']
                #edus = 
                tags = annot2tags(chunk, edus[last_edu:])
                
                for string in converter(filename.replace('data/', ''), chunk):
                    #print(string)
                    if string.startswith('# newdoc id ='):
                        sentence = 0
                        token = 0
                        fo.write(string + '\n')

                    elif string == '\n':
                        fo.write(string)
                        sentence += 1
                        token = 0

                    else:
                        if ' ' in string:
                            string = re.sub(r' .*\t', '\t', string)
                        if 'www' in string:
                            string = re.sub(r'www[^\t]*', '_html_', string)
                        if 'http' in string:
                            string = re.sub(r'http[^ \t]*', '_html_', string)
                            
                        string = prepare_token(string)                        
                        fo.write(string + '\t' + tags[sentence][token] + '\n')
                        
                        if tags[sentence][token] != '_':
                            last_edu += 1
                        
                        token += 1

                    if token == MAX_LEN:
                        print(filename + ' ::: occured very long sentence; truncate to ' + str(MAX_LEN) + ' tokens.')
                        fo.write('\n')
                        sentence += 1
                        token = 0
                        break


preprocess(train)
preprocess(dev, dev=True)
preprocess(test, train=False)

In [ ]:
! ls -laht *.conll

In [ ]:
! mkdir datasets_all

In [ ]:
%%bash -s "$TRAIN_FILE" "$DEV_FILE" "$TEST_FILE"

export TONY_PATH="../tony/"

cp ${1} ${TONY_PATH}/data/rus.rst.rrt/${1}
cp ${2} ${TONY_PATH}/data/rus.rst.rrt/${2}
cp ${3} ${TONY_PATH}/data/rus.rst.rrt/${3}

In [ ]:
! cp -r models/tony_segmentator ../isanlp_rst/models/tony_segmentator